## Note
This notebook serves as a mini tutorial on what you need to fine tune the model on a custom dataset in the simplest way.

The main things that need to be done to do so is to create a dataset class following the implementation of the Coralscapes class and to add the model checkpoint in the arguments of the training script or in the configuration file.

### Running the finetune script on top of this

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Data\ Challenge\ 3\ Group\ 3
!git clone https://github.com/eceo-epfl/coralscapesScripts.git



/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3
fatal: destination path 'coralscapesScripts' already exists and is not an empty directory.


In [ ]:
%cd coralscapesScripts

/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts


In [ ]:
!pip install albumentations transformers torch segmentation_models_pytorch torchmetrics peft matplotlib


# Setup

In [ ]:
import sys
import os

cur = os.path.join(os.getcwd(), '..')
print(cur)
proj_path = os.path.abspath(
    os.path.join(os.getcwd(), '..')
)
coralpath = os.path.join(proj_path, 'coralscapesScripts')
print(coralpath)

sys.path.insert(0, coralpath)
print(
    sys.path
)# # Assume your notebook is in <project_root>/coralscapes/, and coralscapesScripts is in <project_root>/
# project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
# if project_root not in sys.path:
#     sys.path.insert(0, project_root)


/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts/..
/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts
['/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts', '/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython']


In [ ]:
!ls
%cd coralscapesScripts



assets		configs		      environment.yml  README.md
benchmark_runs	coralscapes.egg-info  nbs	       scripts
bin		coralscapesScripts    pyproject.toml
/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts/coralscapesScripts


In [ ]:


import albumentations as A

from torch.utils.data import DataLoader, Dataset
from coralscapesScripts.datasets.dataset import Coralscapes
from coralscapesScripts.datasets.preprocess import get_preprocessor
from coralscapesScripts.datasets.utils import calculate_weights

from coralscapesScripts.segmentation.model import Benchmark_Run
from coralscapesScripts.segmentation.benchmarking import launch_benchmark

from coralscapesScripts.visualization import show_samples

from coralscapesScripts.logger import Logger, save_benchmark_run
from coralscapesScripts.io import setup_config, get_parser, update_config_with_args
import copy

import os

#their eval sucks coz its bloated so using simpligied on
import torch
import torch.nn as nn # Import the neural network module


In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
device_count = torch.cuda.device_count()
for i in range(device_count):
    print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

CUDA Device 0: Tesla T4


device(type='cuda')

## Config

- Currently the Benchmark_Run takes the cfg and pops out the different parts (meaning the cfg file is **changed** after the object initialization).
- If you want to look at the cfg after that, use the cfg_logger copy which will contain the original configuration.
- Keep in mind due to this that if you try to "recreate" the Benchmark_Run object without rerunning the configuration it will most likely result in an error, so better to start over (or at least just rerun the config part of the script).

# dont really get wtvr the fuck they r tryna so imma just roll w it for now
# UPDATE- u just have to rerun the whole thing coz it changes teh cfg. nothing major. ignoe

In [ ]:
cfg = setup_config(config_path='configs/segformer-mit-b2_lora.yaml', config_base_path='configs/base.yaml')

## Args

### Note
- As in the command line, here the arguments take presedence over the config file, so here you can overwrite/test out some parameters without directly changing the cfg.
- Keep in mind not all parameters of the config can be changed through cmd (you can see which ones in the get_parser and update_config_with_args function).

In [ ]:
args_input = "--run-name=fine_tune_notebook --batch-size=4 --batch-size-eval=2 --epochs=50"
args_input = args_input.split(" ")

parser = get_parser()
args = parser.parse_args(args_input)

cfg = update_config_with_args(cfg, args)
cfg_logger = copy.deepcopy(cfg)

# Data

### Note
Make a new dataset class using the Coralscapes class in coralscapesScripts.datasets.dataset as an example. Importantly, it needs to contain the following as arguments
- transform (albumentations)
- transform_target
- N_classes
- id2label  

In [ ]:
root="/content/drive/MyDrive/Data Challenge 3 Group 3/01_data/coral_bleaching/reef_support/UNAL_BLEACHING_TAYRONA"
# root="/content/drive/MyDrive/Data Challenge 3 Group 3/cluster_0"

id2classes = {
    # Background classes -> 0 (background)
    #ignore the label name from here, i jut saw it from the class.json we can play around w it but meh
    0: 0,   # background
    1: 0,   # seagrass
    2: 0,   # sand OR WTVR
    5: 0,   # sand or wtvr
    6: 0,   # rubble or wtvr
    7: 0,   # algae_covered_substrate
    8: 0,   # algae_covered_substrate
    9: 0,   # fish
    10: 0,  # algae_covered_substrate
    11: 0,  # algae_covered_substrate
    12: 0,  # algae_covered_substrate
    13: 0,  # background
    14: 0,  # dark
    15: 0,  # algae_covered_substrate

    # Bleached coral classes -> 1 (bleached)
    3: 1,   # other_coral_bleached
    4: 1,   # other_coral_bleached
    16: 1,  # massive_meandering_bleached -> bleached
    19: 1,  # branching_bleached -> bleached
    33: 1,  # meandering_bleached -> bleached

    # Non-bleached coral classes -> 2 (non-bleached)
    17: 2,  # massive_meandering_alive -> non-bleached
    18: 2,  # rubble (coral rubble) -> non-bleached
    20: 2,  # branching_dead -> non-bleached (dead but not bleached)
    21: 2,  # millepora -> non-bleached
    22: 2,  # branching_alive -> non-bleached
    23: 2,  # massive_meandering_dead -> non-bleached (dead but not bleached)
    24: 2,  # clam -> non-bleached
    25: 2,  # acropora_alive -> non-bleached
    26: 2,  # sea_cucumber -> non-bleached
    27: 2,  # turbinaria -> non-bleached
    28: 2,  # table_acropora_alive -> non-bleached
    29: 2,  # sponge -> non-bleached
    30: 2,  # anemone -> non-bleached
    31: 2,  # pocillopora_alive -> non-bleached
    32: 2,  # table_acropora_dead -> non-bleached (dead but not bleached)
    34: 2,  # stylophora_alive -> non-bleached
    35: 2,  # sea_urchin -> non-bleached
    36: 2,  # meandering_alive -> non-bleached
    37: 2,  # meandering_dead -> non-bleached (dead but not bleached)
    38: 2,  # crown_of_thorn -> non-bleached
    39: 2,  # dead_clam -> non-bleached
}

In [ ]:
# Helper function to map 40-class predictions back to 3 classes
def map_40_to_3_classes(predictions):
    """
    Map 40-class predictions back to 3 classes:
    - 0: Background (classes 0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
    - 1: Bleached (classes 3, 4, 16, 19, 33)
    - 2: Non-bleached (classes 17, 18, 20-32, 34-39)
    """
    # Create a mapping array
    mapping = np.zeros(40, dtype=np.uint8)

    # Set bleached classes
    bleached_classes = [3, 4, 16, 19, 33]
    for cls in bleached_classes:
        mapping[cls] = 1

    # Set non-bleached classes
    non_bleached_classes = [17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39]
    for cls in non_bleached_classes:
        mapping[cls] = 2

    # Apply mapping
    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()

    mapped_predictions = mapping[predictions]
    return mapped_predictions

In [ ]:
for root, dirs, files in os.walk(root):
    for name in dirs:
        print(os.path.join(root, name), 'idoes it exist?')


NameError: name 'root' is not defined

In [ ]:
class CoralBleachingDataset(Dataset):
    """
    Dataset for coral bleaching segmentation.
    Expects data structure:
    - images/: RGB images
    - masks_bleached/: Masks for bleached corals
    - masks_non_bleached/: Masks for non-bleached corals

    This class follows the structure of the original Coralscapes class but works with our 3-class setup.
    """
    def __init__(self, root=root, split='train', transform=None, transform_target=True):
        """
        Args:
            transform (callable): A function/transform that takes in an image and returns a transformed version.
            transform_target (bool): Whether to also transform the segmentation mask.
        """
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.transform_target = transform_target
        self.split = split

        self.CoralClass = namedtuple('CoralClass', ['name', 'id', 'train_id', 'category', 'category_id', 'ignore_in_eval', 'color'])

        self.classes = []
        self.classes.append(self.CoralClass('background', 0, 0, 'background', 0, True, (0, 0, 0)))      # Black - background
        self.classes.append(self.CoralClass('bleached', 1, 1, 'bleached', 1, False, (255, 255, 255)))    # White - bleached coral
        self.classes.append(self.CoralClass('non_bleached', 2, 2, 'non_bleached', 2, False, (128, 128, 128)))  # Gray - non-bleached coral

        # Set number of classes - follow the same logic as the original Coralscapes class
        self.N_classes = 3  # We have 3 classes: background, bleached, non-bleached

        self.id2label = {0: "background", 1: "bleached", 2: "non_bleached"}
        self.label2id = {v: k for k, v in self.id2label.items()}

        self.train_id_to_color = np.array([c.color for c in self.classes])

        # juliette double check these.
        # Define mapping from original 40 classes to our 3 classes
        # This mapping is critical for interfacing with the model
        self.class_mapping = {
            # Background classes -> 0
            0: 0, 1: 0, 2: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0,
            # Bleached coral classes -> 1
            3: 1, 4: 1, 16: 1, 19: 1, 33: 1,
            # Non-bleached coral classes -> 2
            17: 2, 18: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2,
            30: 2, 31: 2, 32: 2, 34: 2, 35: 2, 36: 2, 37: 2, 38: 2, 39: 2
        }
        # Create reverse mapping for model output interpretation
        self.reverse_class_mapping = {}
        for original_class, our_class in self.class_mapping.items():
            if our_class not in self.reverse_class_mapping:
                self.reverse_class_mapping[our_class] = []
            self.reverse_class_mapping[our_class].append(original_class)

        # Create train/val/test splits
        self.images_dir = os.path.join(self.root, 'images')
        all_images = [f for f in os.listdir(self.images_dir)
                     if f.endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'))]

        np.random.seed(42)
        np.random.shuffle(all_images)

        if split == 'train':
            self.images = all_images[:int(0.7*len(all_images))]
        elif split == 'val':
            self.images = all_images[int(0.7*len(all_images)):int(0.85*len(all_images))]
        elif split == 'test':
            self.images = all_images[int(0.85*len(all_images)):]

        print(f"Loaded {len(self.images)} images for {split} split")

    def __getitem__(self, index):
        """
        Retrieve and transform an image and its corresponding segmentation maps.
        """
        img_name = self.images[index]
        image_path = os.path.join(self.root, 'images', img_name)

        # Get the base name without extension
        base_name = os.path.splitext(img_name)[0]

        # Load image
        image = np.array(Image.open(image_path).convert('RGB'))

        # Load bleached and non-bleached masks
        bleached_mask_path = os.path.join(self.root, 'masks_bleached', f"{base_name}_bleached.png")
        non_bleached_mask_path = os.path.join(self.root, 'masks_non_bleached', f"{base_name}_non_bleached.png")

        try:
            bleached_mask = np.array(Image.open(bleached_mask_path).convert('L'))
            non_bleached_mask = np.array(Image.open(non_bleached_mask_path).convert('L'))
        except FileNotFoundError as e:
            print(f"Error loading mask for {img_name}: {e}")
            raise

        # Create a combined mask where:
        # 0 = background, 1 = bleached, 2 = non-bleached
        target = np.zeros_like(bleached_mask, dtype=np.uint8)
        target[bleached_mask > 128] = 1      # White pixels in bleached mask -> class 1
        target[non_bleached_mask > 128] = 2  # White pixels in non-bleached mask -> class 2
        # Background remains 0 (black pixels)

        # ADD THIS: Resize both image and mask to the same size
        target_size = (512, 512)  # or whatever size you want

        # Resize image
        image_pil = Image.fromarray(image)
        image_resized = image_pil.resize(target_size)
        image = np.array(image_resized)

        # Resize mask
        target_pil = Image.fromarray(target)
        target_resized = target_pil.resize(target_size, Image.NEAREST)  # Use NEAREST for masks
        target = np.array(target_resized)

        if self.transform:
            if self.transform_target:
                transformed = self.transform(image=image, mask=target)
                image = transformed["image"]
                target = transformed["mask"]
            else:
                transformed = self.transform(image=image)
                image = transformed["image"]

        # Convert image to PyTorch format (C, H, W) if it's still a numpy array
        if isinstance(image, np.ndarray):
            image = image.transpose(2, 0, 1)

        return image, target

    def __len__(self):
        return len(self.images)

NameError: name 'root' is not defined

In [ ]:
#resizing ffs coz images are shite.
transforms = {}
for split in cfg.augmentation:
    transform_list = [A.Resize(height=512, width=512)] # start with resize to ensure consistent size

    for transform_name, transform_params in cfg.augmentation[split].items():
        if "Crop" in transform_name and "height" in transform_params and "width" in transform_params:# skipping any crop operations that would be larger than our resize
            if transform_params["height"] > 512 or transform_params["width"] > 512:
                print(f"Warning: Adjusting '{transform_name}' to match 512x512 size in split '{split}'")
                transform_params["height"] = min(transform_params["height"], 512)
                transform_params["width"] = min(transform_params["width"], 512)

        transform_list.append(getattr(A, transform_name)(**transform_params))

    transforms[split] = A.Compose(transform_list)

In [ ]:
cfg.data.root = '/content/drive/MyDrive/Data Challenge 3 Group 3/01_data/coral_bleaching/reef_support/UNAL_BLEACHING_TAYRONA'
  # Adjust this path to your data directory FOR EACH OF THE CLUSTER/ this is v important or wasted gpu hours...
# cfg.data.root="/content/drive/MyDrive/Data Challenge 3 Group 3/cluster_0"


In [ ]:
train_dataset = CoralBleachingDataset(root=cfg.data.root, split='train', transform=transforms["train"])

transform_target = cfg.training.eval.transform_target if cfg.training.eval is not None and cfg.training.eval.transform_target is not None else True

val_dataset = CoralBleachingDataset(root=cfg.data.root, split='val', transform=transforms["val"], transform_target=transform_target)
test_dataset = CoralBleachingDataset(root=cfg.data.root, split='test', transform=transforms["test"], transform_target=transform_target)

print(f"Classes: {train_dataset.id2label}")
print(f"Number of classes: {train_dataset.N_classes}")
print(f"Train dataset has {len(train_dataset)} samples")
print(f"Validation dataset has {len(val_dataset)} samples")
print(f"Test dataset has {len(test_dataset)} samples")

Loaded 460 images for train split
Loaded 99 images for val split
Loaded 99 images for test split
Classes: {0: 'background', 1: 'bleached', 2: 'non_bleached'}
Number of classes: 3
Train dataset has 460 samples
Validation dataset has 99 samples
Test dataset has 99 samples


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=cfg.data.batch_size, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=cfg.data.batch_size_eval, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=cfg.data.batch_size_eval, shuffle=False, num_workers=2)

In [ ]:
print(len(train_loader), len(val_loader), len(test_loader))

115 50 50


In [ ]:
weight = calculate_weights(train_dataset).to(device) if(cfg.data.weight) else None

# Some masks folder sanity checks

## Visualizations

In [ ]:
# show_samples(train_dataset, n=2)
# wont run coz i changed the class to coralscapebleachingdataset but can be visualized another way, not important yet.

In [ ]:
# show_samples(val_dataset, n=2)

In [ ]:
# show_samples(test_dataset, n=2)

# Benchmarking

## Model

In [ ]:
benchmark_run = Benchmark_Run(run_name=cfg.run_name,
                             model_name=cfg.model.name,
                             N_classes=3,  # Using 40 for model compatibility
                             device=device,
                             model_kwargs=cfg.model.kwargs,
                             model_checkpoint=cfg.model.checkpoint,
                             lora_kwargs=cfg.lora,
                             training_hyperparameters=cfg.training)

/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts/coralscapesScripts/datasets/preprocess.py:105: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'ignore_index', 'num_labels'
  preprocessor = SegformerImageProcessor(**default_preprocessor_kwargs)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/99.0M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
benchmark_run.print_trainable_parameters()

trainable params: 5290883 || all params: 32631494 || trainable%: 16.21


### Note
- If you want to use a LoRA approach, as long as you have it in the config (as in this case) with the correct parameters, you are good to go to the training script, no other changes need to be made.  
- If you want to freeze a specific layer, the easiest start is to just look at the model here and freeze the parameters you prefer.

In [ ]:
benchmark_run.model

PeftModel(
  (base_model): LoraModel(
    (model): SegformerForSemanticSegmentation(
      (segformer): SegformerModel(
        (encoder): SegformerEncoder(
          (patch_embeddings): ModuleList(
            (0): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
              (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
            )
            (1): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            )
            (2): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
              (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
            )
            (3): SegformerOverlapPatchEmbeddings(
              (proj): Conv2d(320, 512, kernel_si

## Logger

In [ ]:
# !pip install --upgrade wandb


In [ ]:
logger = Logger(
    project = cfg.logger.wandb_project,
    benchmark_run = benchmark_run,
    log_epochs = cfg.logger.log_epochs,
    config = cfg_logger,
    checkpoint_dir ="old/coralscapesScripts/"
)
# wandb api key if they ask for it, put this 9770552259c029b0835547eed1ed7173e8c8eccf

wandb: Currently logged in as: muhammadibnerafiq (muhammadibnerafiq-tilburg-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Run

In [ ]:
benchmark_run.print_trainable_parameters()

trainable params: 5290883 || all params: 32631494 || trainable%: 16.21


In [ ]:
data_batch = next(iter(train_loader))
labels_batch = data_batch[1] # Access labels using index 1

print("Shape of labels batch:", labels_batch.shape)
print("Unique values in labels batch:", torch.unique(labels_batch))

Shape of labels batch: torch.Size([4, 512, 512])
Unique values in labels batch: tensor([0, 1, 2], dtype=torch.uint8)


In [ ]:
# benchmark_metrics = launch_benchmark(train_loader, val_loader, test_loader, benchmark_run, logger = logger)
# save_benchmark_run(benchmark_run, benchmark_metrics)

YAY NOW K-FOLD IS BACK MOFOS
*   List item
*   List item



In [ ]:
# chatgpt version of k-fold should be used ideally
from sklearn.model_selection import KFold

def calculate_iou(pred, target, num_classes=3):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds & target_inds).sum().float().item()
        union = (pred_inds | target_inds).sum().float().item()
        iou = intersection / union if union > 0 else 0
        ious.append(iou)

    return ious, np.mean(ious)

def train_and_validate_kfold(train_loader, val_loader, benchmark_run, criterion, n_splits=5, logger=None,
                            start_epoch=0, end_epoch=None, save_dir="./checkpoints_og_reefsupport", save_best_only=True):
    """
    Perform k-fold cross-validation training and validation.

    Args:
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data
        benchmark_run: Object containing model, optimizer, device, etc.
        criterion: Loss function
        n_splits: Number of folds for k-fold cross-validation
        logger: Optional logger for metrics
        start_epoch: Starting epoch number
        end_epoch: Ending epoch number
        save_dir: Directory to save model checkpoints
        save_best_only: If True, only save model when validation performance improves
    """
    if end_epoch is None:
        end_epoch = benchmark_run.training_hyperparameters.epochs

    # Create save directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Store metrics for each fold
    fold_metrics = []

    # Get the dataset from the train loader
    train_dataset = train_loader.dataset
    val_dataset = val_loader.dataset

    # Combine datasets for k-fold splitting
    # If they're the same dataset with different samplers, just use one
    if train_dataset == val_dataset:
        full_dataset = train_dataset
    else:
        # This assumes both datasets have the same structure and can be concatenated
        try:
            full_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])
        except:
            print("Warning: Could not concatenate datasets. Using training dataset for k-fold.")
            full_dataset = train_dataset

    # Initialize k-fold cross validation
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Get batch size and num_workers from original loaders
    batch_size = train_loader.batch_size
    num_workers = train_loader.num_workers

    # Track the best overall model across all folds based on the custom score
    best_overall_score = -1.0
    best_overall_fold = -1
    best_overall_epoch = -1
    best_overall_metrics = None
    best_overall_model_state_dict = None


    # Loop through each fold
    for fold, (train_idx, val_idx) in enumerate(kfold.split(range(len(full_dataset)))):
        print(f"\n{'='*20} FOLD {fold+1}/{n_splits} {'='*20}")

        # Create data loaders for this fold
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

        fold_train_loader = torch.utils.data.DataLoader(
            full_dataset,
            batch_size=batch_size,
            sampler=train_subsampler,
            num_workers=num_workers
        )

        fold_val_loader = torch.utils.data.DataLoader(
            full_dataset,
            batch_size=batch_size,
            sampler=val_subsampler,
            num_workers=num_workers
        )

        # Reset model weights for each fold if possible
        if hasattr(benchmark_run.model, 'reset_parameters'):
            benchmark_run.model.reset_parameters()
        else:
            print("Warning: Model does not have reset_parameters method. Using current weights.")

        # Reset optimizer for the new model parameters
        if hasattr(benchmark_run, 'optimizer_class') and hasattr(benchmark_run, 'optimizer_params'):
            benchmark_run.optimizer = benchmark_run.optimizer_class(
                benchmark_run.model.parameters(),
                **benchmark_run.optimizer_params
            )

        # Initialize metrics for this fold
        fold_benchmark_metrics = {
            "train_loss": [],
            "val_loss": [],
            "val_accuracy": [],
            "val_miou": [],
            "val_iou_background": [],
            "val_iou_bleached": [],
            "val_iou_non_bleached": [],
            "val_custom_score": []
        }

        # Track best performance for this fold
        best_fold_score = -1.0

        # Train for specified number of epochs
        for epoch in range(start_epoch, end_epoch):
            print(f"EPOCH {epoch+1}/{end_epoch}:")

            # Train
            benchmark_run.model.train()
            train_loss = 0.0
            num_batches = 0

            for data in tqdm(fold_train_loader, desc="Training"):
                # Move data to device
                inputs, labels = data
                inputs = inputs.to(benchmark_run.device)
                labels = labels.to(benchmark_run.device)

                # Zero the parameter gradients
                benchmark_run.optimizer.zero_grad()

                # Forward pass
                outputs = benchmark_run.model(inputs)

                # Resize model output (logits) to match target size
                resized_logits = F.interpolate(
                    outputs.logits,
                    size=labels.shape[-2:],
                    mode='bilinear',
                    align_corners=False
                )

                # Calculate loss
                loss = criterion(resized_logits, labels.long())

                # Backward pass and optimize
                loss.backward()
                benchmark_run.optimizer.step()

                train_loss += loss.item()
                num_batches += 1

            if num_batches > 0:
                train_loss /= num_batches

            fold_benchmark_metrics["train_loss"].append(train_loss)

            if hasattr(benchmark_run, "scheduler"):
                benchmark_run.scheduler.step()

            print(f'LOSS train {train_loss:.4f}')

            # Validation
            benchmark_run.model.eval()
            val_loss = 0.0
            num_batches_val = 0
            val_correct = 0
            val_total = 0
            val_ious = np.zeros(3)  # For 3 classes

            with torch.no_grad():
                for vdata in tqdm(fold_val_loader, desc="Validating"):
                    inputs, labels = vdata
                    inputs = inputs.to(benchmark_run.device)
                    labels = labels.to(benchmark_run.device)

                    # Forward pass
                    outputs = benchmark_run.model(inputs)

                    # Resize model output (logits) to match target size
                    resized_logits = F.interpolate(
                        outputs.logits,
                        size=labels.shape[-2:],
                        mode='bilinear',
                        align_corners=False
                    )

                    # Calculate loss
                    loss = criterion(resized_logits, labels.long())

                    # Calculate accuracy
                    preds = torch.argmax(resized_logits, dim=1)
                    val_correct += (preds == labels).sum().item()
                    val_total += labels.numel()

                    # Calculate IoU
                    batch_ious, _ = calculate_iou(preds, labels)
                    val_ious += np.array(batch_ious)

                    val_loss += loss.item()
                    num_batches_val += 1

            if num_batches_val > 0:
                val_loss /= num_batches_val
                val_ious /= num_batches_val

            val_accuracy = val_correct / val_total if val_total > 0 else 0
            val_miou = np.mean(val_ious)

            # Calculate custom ranking score
            # rank = alpha * accuracy + beta * mIoU
            # mIoU = 0.1 * iou_background + 0.45 * iou_bleached + 0.45 * iou_non_bleached
            alpha = 0.2
            beta = 0.8
            miou_weighted = (0.1 * val_ious[0] + 0.45 * val_ious[1] + 0.45 * val_ious[2])
            custom_score = alpha * val_accuracy + beta * miou_weighted

            # Store metrics
            fold_benchmark_metrics["val_loss"].append(val_loss)
            fold_benchmark_metrics["val_accuracy"].append(val_accuracy)
            fold_benchmark_metrics["val_miou"].append(val_miou)
            fold_benchmark_metrics["val_iou_background"].append(val_ious[0])
            fold_benchmark_metrics["val_iou_bleached"].append(val_ious[1])
            fold_benchmark_metrics["val_iou_non_bleached"].append(val_ious[2])
            fold_benchmark_metrics["val_custom_score"].append(custom_score)


            print(f'LOSS valid {val_loss:.4f}')
            print(f'Accuracy valid {val_accuracy:.4f}')
            print(f'mIoU valid {val_miou:.4f}')
            print(f'IoU per class: Background={val_ious[0]:.4f}, Bleached={val_ious[1]:.4f}, Non-bleached={val_ious[2]:.4f}')
            print(f'Custom Score valid {custom_score:.4f}')


            # Log metrics if logger is provided
            if logger and epoch % logger.log_epochs == 0:
                logger.log({
                    f"fold_{fold+1}/train/loss": train_loss,
                    f"fold_{fold+1}/val/loss": val_loss,
                    f"fold_{fold+1}/val/accuracy": val_accuracy,
                    f"fold_{fold+1}/val/miou": val_miou,
                    f"fold_{fold+1}/val/iou_background": val_ious[0],
                    f"fold_{fold+1}/val/iou_bleached": val_ious[1],
                    f"fold_{fold+1}/val/iou_non_bleached": val_ious[2],
                    f"fold_{fold+1}/val/custom_score": custom_score
                }, epoch)

            # Save best model for this fold based on custom score
            if save_best_only:
                if custom_score > best_fold_score:
                    best_fold_score = custom_score
                    checkpoint_path = os.path.join(save_dir, f"model_fold_{fold+1}_best_score.pth")
                    torch.save(benchmark_run.model.state_dict(), checkpoint_path)
                    print(f"Saved improved model checkpoint to {checkpoint_path} (Custom Score: {custom_score:.4f})")

            # Check if this is the best overall model
            if custom_score > best_overall_score:
                best_overall_score = custom_score
                best_overall_fold = fold + 1
                best_overall_epoch = epoch + 1
                best_overall_metrics = {
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "val_accuracy": val_accuracy,
                    "val_miou": val_miou,
                    "val_iou_background": val_ious[0],
                    "val_iou_bleached": val_ious[1],
                    "val_iou_non_bleached": val_ious[2],
                    "val_custom_score": custom_score
                }
                best_overall_model_state_dict = copy.deepcopy(benchmark_run.model.state_dict())


        # Save metrics for this fold
        fold_metrics.append(fold_benchmark_metrics)

        # Save fold metrics
        metrics_path = os.path.join(save_dir, f"metrics_fold_{fold+1}.json")
        with open(metrics_path, "w") as f:
            json.dump(fold_benchmark_metrics, f)

    # Calculate and save average metrics across all folds
    avg_metrics = {
        "avg_val_miou_final": np.mean([metrics["val_miou"][-1] for metrics in fold_metrics]),
        "avg_val_accuracy_final": np.mean([metrics["val_accuracy"][-1] for metrics in fold_metrics]),
        "avg_val_iou_background_final": np.mean([metrics["val_iou_background"][-1] for metrics in fold_metrics]),
        "avg_val_iou_bleached_final": np.mean([metrics["val_iou_bleached"][-1] for metrics in fold_metrics]),
        "avg_val_iou_non_bleached_final": np.mean([metrics["val_iou_non_bleached"][-1] for metrics in fold_metrics]),
        "avg_val_custom_score_final": np.mean([metrics["val_custom_score"][-1] for metrics in fold_metrics]),
        "best_val_miou_across_folds": np.max([np.max(metrics["val_miou"]) for metrics in fold_metrics]),
        "best_val_custom_score_across_folds": np.max([np.max(metrics["val_custom_score"]) for metrics in fold_metrics]),
        "fold_metrics": fold_metrics,
        "best_overall_model": {
            "fold": best_overall_fold,
            "epoch": best_overall_epoch,
            "metrics": best_overall_metrics
        }
    }

    # Save the best overall model checkpoint
    if best_overall_model_state_dict is not None:
        best_overall_checkpoint_path = os.path.join(save_dir, "best_overall_model.pth")
        torch.save(best_overall_model_state_dict, best_overall_checkpoint_path)
        print(f"\nSaved best overall model checkpoint to {best_overall_checkpoint_path} (Custom Score: {best_overall_score:.4f})")


    # Save overall metrics
    overall_metrics_path = os.path.join(save_dir, "kfold_metrics.json")
    with open(overall_metrics_path, "w") as f:
        json.dump(avg_metrics, f)

    print(f"\nAverage Final mIoU across folds: {avg_metrics['avg_val_miou_final']:.4f}")
    print(f"Best mIoU across all folds: {avg_metrics['best_val_miou_across_folds']:.4f}")
    print(f"Average Final Custom Score across folds: {avg_metrics['avg_val_custom_score_final']:.4f}")
    print(f"Best Custom Score across all folds: {avg_metrics['best_val_custom_score_across_folds']:.4f}")
    print(f"\nBest overall model found in Fold {best_overall_fold}, Epoch {best_overall_epoch} with Custom Score: {best_overall_score:.4f}")
    print("="*50)

    return avg_metrics

In [ ]:
criterion = nn.CrossEntropyLoss(weight=weight) # Use the calculated weight

# benchmark_metrics = train_and_validate_simplified(
#     train_loader,
#     val_loader,
#     benchmark_run,
#     criterion=criterion,
#     logger=logger,
#     save_dir=save_directory,
#     save_epochs=save_frequency_epochs # Pass the save frequency
# )
benchmark_metrics = train_and_validate_kfold(
    train_loader,
    val_loader,
    benchmark_run,
    criterion,
    n_splits=5,
    save_best_only=True
)


==================== FOLD 1/5 ====================
EPOCH 1/50:


Training: 100%|██████████| 112/112 [04:31<00:00,  2.42s/it]


LOSS train 0.6733


Validating: 100%|██████████| 28/28 [01:03<00:00,  2.28s/it]


LOSS valid 0.6194
Accuracy valid 0.7715
mIoU valid 0.5304
IoU per class: Background=0.7591, Bleached=0.6120, Non-bleached=0.2201
Custom Score valid 0.5146
Saved improved model checkpoint to ./checkpoints_og_reefsupport/model_fold_1_best_score.pth (Custom Score: 0.5146)
EPOCH 2/50:


...

LOSS valid 0.2105
Accuracy valid 0.9133
mIoU valid 0.7815
IoU per class: Background=0.8779, Bleached=0.7797, Non-bleached=0.6869
Custom Score valid 0.7809
EPOCH 50/50:


Training: 100%|██████████| 112/112 [00:44<00:00,  2.54it/s]


LOSS train 0.1495


Validating: 100%|██████████| 28/28 [00:10<00:00,  2.76it/s]


LOSS valid 0.1824
Accuracy valid 0.9276
mIoU valid 0.8269
IoU per class: Background=0.8990, Bleached=0.8175, Non-bleached=0.7640
Custom Score valid 0.8268

Saved best overall model checkpoint to ./checkpoints_og_reefsupport/best_overall_model.pth (Custom Score: 0.8517)

Average Final mIoU across folds: 0.7495
Best mIoU across all folds: 0.8520
Average Final Custom Score across folds: 0.7473
Best Custom Score across all folds: 0.8517

Best overall model found in Fold 5, Epoch 2 with Custom Score: 0.8517


**Inference Elvir**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Data\ Challenge\ 3\ Group\ 3
!git clone https://github.com/eceo-epfl/coralscapesScripts.git
%cd coralscapesScripts
!pip install albumentations transformers torch segmentation_models_pytorch torchmetrics peft matplotlib
import sys
import os
cur = os.path.join(os.getcwd(), '..')
print(cur)
proj_path = os.path.abspath(
    os.path.join(os.getcwd(), '..')
)
coralpath = os.path.join(proj_path, 'coralscapesScripts')
print(coralpath)
sys.path.insert(0, coralpath)
print(
    sys.path
)
!ls
%cd coralscapesScripts
from coralscapesScripts.datasets.dataset import Coralscapes
from coralscapesScripts.datasets.preprocess import get_preprocessor
from coralscapesScripts.datasets.utils import calculate_weights
from coralscapesScripts.segmentation.model import Benchmark_Run
from coralscapesScripts.segmentation.benchmarking import launch_benchmark
from coralscapesScripts.visualization import show_samples
from coralscapesScripts.logger import Logger, save_benchmark_run
from coralscapesScripts.io import setup_config, get_parser, update_config_with_args
import copy
from collections import namedtuple
import json
import torch

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
device_count = torch.cuda.device_count()
for i in range(device_count):
    print(f"CUDA Device {i}: {torch.cuda.get_device_name(i)}")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device
cfg = setup_config(config_path='configs/segformer-mit-b2_lora.yaml', config_base_path='configs/base.yaml')
args_input = "--run-name=fine_tune_notebook --batch-size=4 --batch-size-eval=2 --epochs=50"
args_input = args_input.split(" ")
parser = get_parser()
args = parser.parse_args(args_input)
cfg = update_config_with_args(cfg, args)
cfg_logger = copy.deepcopy(cfg)

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3
fatal: destination path 'coralscapesScripts' already exists and is not an empty directory.
/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 32.7 MB/s eta 0:00:00
/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts/..
/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts
['/content/drive/.shortcut-targets-by-id/15pPCEVRFyHb3JQkFSnP8L6a0_nfm8ijp/Data Challenge 3 Group 3/coralscapesScripts', '/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-pa

In [2]:

import os
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset
from torchmetrics.classification import MulticlassJaccardIndex, MulticlassAccuracy
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import albumentations as A
import cv2
from transformers import SegformerConfig, SegformerForSemanticSegmentation
from peft import PeftModel, LoraConfig
import pandas as pd
import math
import random

N_CLASSES = 3
MODEL_NAME = "nvidia/mit-b2"
INPUT_SIZE = (512, 512)

In [3]:
class CoralBleachingDataset(Dataset):

    def __init__(self, root, transform=None):
        self.root = os.path.expanduser(root)
        self.transform = transform

        self.images_dir = os.path.join(self.root, 'images')
        self.masks_bleached_dir = os.path.join(self.root, 'masks_bleached')
        self.masks_non_bleached_dir = os.path.join(self.root, 'masks_non_bleached')

        if not os.path.isdir(self.images_dir):
            raise FileNotFoundError(f"Image directory not found: {self.images_dir}")

        all_images = [f for f in os.listdir(self.images_dir)
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        self.images = []

        for img_name in all_images:
            base_name, ext = os.path.splitext(img_name)

            bleached_mask_name = f"{base_name}_bleached.png"
            non_bleached_mask_name = f"{base_name}_non_bleached.png"

            bleached_mask_path = os.path.join(self.masks_bleached_dir, bleached_mask_name)
            non_bleached_mask_path = os.path.join(self.masks_non_bleached_dir, non_bleached_mask_name)

            if os.path.exists(bleached_mask_path) and os.path.exists(non_bleached_mask_path):
                 self.images.append(img_name)

        print(f"Found {len(self.images)} image-mask pairs for inference.")
        if not self.images:
            print(f"CRITICAL: Found 0 pairs in {self.root}. Check naming and file extensions.")

    def __getitem__(self, index):
        img_name = self.images[index]
        base_name, ext = os.path.splitext(img_name)

        image_path = os.path.join(self.root, 'images', img_name)
        bleached_mask_path = os.path.join(self.root, 'masks_bleached', f"{base_name}_bleached.png")
        non_bleached_mask_path = os.path.join(self.root, 'masks_non_bleached', f"{base_name}_non_bleached.png")

        image = np.array(Image.open(image_path).convert('RGB'))
        bleached_mask = np.array(Image.open(bleached_mask_path).convert('L'))
        non_bleached_mask = np.array(Image.open(non_bleached_mask_path).convert('L'))

        target = np.zeros_like(bleached_mask, dtype=np.uint8)
        target[non_bleached_mask > 128] = 2
        target[bleached_mask > 128] = 1

        if self.transform:
            transformed = self.transform(image=image, mask=target)
            image = transformed["image"]
            target = transformed["mask"]

        image = image.transpose(2, 0, 1)
        image = torch.tensor(image, dtype=torch.float32) / 255.0
        target = torch.tensor(target, dtype=torch.long)

        return image, target, img_name

    def __len__(self):
        return len(self.images)

def load_lora_segformer_model(checkpoint_path, model_name, n_classes, device):
    config = SegformerConfig.from_pretrained(model_name, num_labels=n_classes)
    model = SegformerForSemanticSegmentation.from_pretrained(
        model_name, config=config, ignore_mismatched_sizes=True
    ).to(device)

    lora_config = LoraConfig(r=128, lora_alpha=16, target_modules=["query", "value"],
                             lora_dropout=0.1, bias="none", modules_to_save=["decode_head"])

    model = PeftModel(model, lora_config)

    try:
        checkpoint = torch.load(checkpoint_path, map_location=device)
        state_dict_to_load = checkpoint.get('model_state_dict', checkpoint)
        model.load_state_dict(state_dict_to_load, strict=False)
    except Exception as e:
        print(f"FATAL ERROR during model loading for {checkpoint_path}: {e}")
        return None

    model.to(device)
    model.eval()
    return model

def run_inference_and_metrics(model, dataloader, device, n_classes):

    jaccard = MulticlassJaccardIndex(num_classes=n_classes, average='macro', ignore_index=None).to(device)
    accuracy = MulticlassAccuracy(num_classes=n_classes, average='macro', ignore_index=None).to(device)
    jaccard_per_class = MulticlassJaccardIndex(num_classes=n_classes, average='none', ignore_index=None).to(device)

    model.eval()
    with torch.no_grad():
        for inputs, labels, _ in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            resized_logits = F.interpolate(outputs.logits, size=labels.shape[-2:], mode='bilinear', align_corners=False)
            preds = torch.argmax(resized_logits, dim=1)

            jaccard.update(preds, labels)
            accuracy.update(preds, labels)
            jaccard_per_class.update(preds, labels)

    results = {
        "Accuracy": accuracy.compute().item(),
        "mIoU": jaccard.compute().item(),
        "IoU - Background": jaccard_per_class.compute()[0].item(),
        "IoU - Bleached": jaccard_per_class.compute()[1].item(),
        "IoU - Non-Bleached": jaccard_per_class.compute()[2].item()
    }

    return results

def calculate_custom_metrics(metrics):
    acc = metrics['Accuracy']
    iou_bg = metrics['IoU - Background']
    iou_bl = metrics['IoU - Bleached']
    iou_nbl = metrics['IoU - Non-Bleached']
    mIoU_weighted = (0.1 * iou_bg) + (0.45 * iou_bl) + (0.45 * iou_nbl)
    coral_rank_score = (0.2 * acc) + (0.8 * mIoU_weighted)
    return mIoU_weighted, coral_rank_score

In [4]:
def visualize_single_sample_canvas(model, full_dataset, index, run_name, vis_dir):

    COLOR_MAP_RGB = np.array([
        [0, 0, 0],
        [255, 153, 153],
        [229, 255, 204]
    ], dtype=np.uint8)

    COLOR_MAP_RGBA_OVERLAY = np.array([
        [0, 0, 0, 0],
        [255, 153, 153, 180],
        [229, 255, 204, 180]
    ], dtype=np.uint8)

    IMAGE_W, IMAGE_H = INPUT_SIZE
    MARGIN = 60
    PANEL_GAP = 40
    LEGEND_HEIGHT = 80

    try:
        font_path_title = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
        font_path_legend = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
        font_size_title = 45
        font_size_legend = 35
        font_title = ImageFont.truetype(font_path_title, font_size_title)
        font_legend = ImageFont.truetype(font_path_legend, font_size_legend)
    except IOError:
        font_size_title = 35
        font_size_legend = 25
        font_title = ImageFont.load_default()
        font_legend = ImageFont.load_default()
        print(f"  Warning: mwah, fond not found. Using default font with size {font_size_title} for tltles ")


    CANVAS_W = (IMAGE_W * 4) + (PANEL_GAP * 3) + (MARGIN * 2)
    CANVAS_H = IMAGE_H + (MARGIN * 2) + LEGEND_HEIGHT

    CANVAS_COLOR = (255, 255, 255)

    image_tensor, label_tensor, img_name = full_dataset[index]
    inputs = image_tensor.unsqueeze(0).to(DEVICE)

    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        resized_logits = F.interpolate(
            outputs.logits, size=label_tensor.shape[-2:], mode='bilinear', align_corners=False
        )
        preds = torch.argmax(resized_logits, dim=1)

    image_np = (image_tensor.cpu().numpy().transpose(1, 2, 0) * 255).astype(np.uint8)

    label_indices = label_tensor.cpu().numpy()
    gt_mask_color_np = COLOR_MAP_RGB[label_indices]

    pred_indices = preds[0].cpu().numpy()
    pred_mask_color_np = COLOR_MAP_RGB[pred_indices]

    overlay_image_pil = Image.fromarray(image_np).convert("RGBA")
    overlay_mask_rgba = COLOR_MAP_RGBA_OVERLAY[pred_indices]
    overlay_mask_pil = Image.fromarray(overlay_mask_rgba)
    final_overlay_pil = Image.alpha_composite(overlay_image_pil, overlay_mask_pil)
    final_overlay_np = np.array(final_overlay_pil.convert("RGB"))

    panels = [
        Image.fromarray(image_np),
        Image.fromarray(gt_mask_color_np),
        Image.fromarray(pred_mask_color_np),
        Image.fromarray(final_overlay_np)
    ]

    canvas = Image.new('RGB', (CANVAS_W, CANVAS_H), color=CANVAS_COLOR)
    draw = ImageDraw.Draw(canvas)

    titles = ["Input Image", "Ground Truth Mask", "Prediction Mask", "Prediction Overlay"]
    text_color = (0, 0, 0)

    current_x = MARGIN

    for idx, panel in enumerate(panels):

        title = titles[idx]

        text_w = draw.textlength(title, font=font_title)

        title_x = current_x + (IMAGE_W - text_w) / 2
        try:
            _, _, _, title_bbox_h = font_title.getbbox(title)
            title_y = (MARGIN - title_bbox_h) / 2
        except:
            title_y = (MARGIN - font_size_title) / 2

        draw.text((title_x, title_y), title, font=font_title, fill=text_color)

        image_y = MARGIN
        canvas.paste(panel, (current_x, image_y))

        current_x += IMAGE_W + PANEL_GAP

    legend_items = [
        ("Background (Class 0)", COLOR_MAP_RGB[0]),
        ("Bleached Coral (Class 1)", COLOR_MAP_RGB[1]),
        ("Non-Bleached Coral (Class 2)", COLOR_MAP_RGB[2])
    ]

    legend_title = "LEGEND:"
    color_box_size = 30
    spacing = 50

    total_legend_width = draw.textlength(legend_title, font=font_legend) + 30
    for text, _ in legend_items:
        total_legend_width += color_box_size + 10
        total_legend_width += draw.textlength(text, font=font_legend) + spacing
    total_legend_width -= spacing

    legend_start_x = (CANVAS_W - total_legend_width) / 2

    text_bbox = font_legend.getbbox("A")
    text_height = text_bbox[3] - text_bbox[1]

    legend_y = CANVAS_H - LEGEND_HEIGHT + (LEGEND_HEIGHT - text_height) / 2

    current_x = legend_start_x

    draw.text((current_x, legend_y), legend_title, font=font_legend, fill=text_color)
    current_x += draw.textlength(legend_title, font=font_legend) + 20

    for text, color_rgb in legend_items:
        rect_y_start = legend_y - 2
        draw.rectangle([current_x, rect_y_start, current_x + color_box_size, rect_y_start + color_box_size], fill=tuple(color_rgb))
        current_x += color_box_size + 10
        draw.text((current_x, legend_y), text, font=font_legend, fill=text_color)
        current_x += draw.textlength(text, font=font_legend) + spacing

    clean_run_name = run_name.replace(" ", "_").replace("(", "").replace(")", "")
    output_filename = f"{clean_run_name}_sample_canvas_{os.path.splitext(img_name)[0]}.png"
    canvas.save(os.path.join(vis_dir, output_filename))

    print(f"  -> Saved final canvas visualization for: {run_name} ({img_name})")

In [5]:
DRIVE_BASE_PATH = "/content/drive/MyDrive/"
PROJECT_DIR = os.path.join(DRIVE_BASE_PATH, 'Data Challenge 3 Group 3')

CHECKPOINT_BASE_DIR = os.path.join(
    PROJECT_DIR,
    'coralscapesScripts',
    'coralscapesScripts'
)
VIS_DIR = os.path.join(PROJECT_DIR, 'inference_visualization_final_layout')
os.makedirs(VIS_DIR, exist_ok=True)
print(f"Final canvas visualizations will be saved to: {VIS_DIR}")

BATCH_SIZE = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

inference_runs = [
    ('Cyan 0', 'data_cyan_0', 'checkpoints_cluster_0_cyan'),
    ('Cyan 1', 'data_cyan_1', 'checkpoints_cluster_1_cyan'),
    ('Cyan 2', 'data_cyan_2', 'checkpoints_cluster_2_cyan'),
    ('Magenta 0', 'data_magenta_0', 'checkpoints_cluster_0_magenta'),
    ('Magenta 1', 'data_magenta_1', 'checkpoints_cluster_1_magenta'),
    ('Magenta 2', 'data_magenta_2', 'checkpoints_cluster_2_magenta'),
    ('Yellow 0', 'data_yellow_0', 'checkpoints_yellow_0'),
    ('Yellow 1', 'data_yellow_1', 'checkpoints_yellow_1'),
    ('Yellow 2', 'data_yellow_2', 'checkpoints_yellow_2'),
    ('Cluster 0 (Base)', 'cluster_0', 'checkpoints_cluster_0_base_model'),
    ('Cluster 1 (Base)', 'cluster_1', 'checkpoints_cluster_1_base_model'),
    ('Cluster 2 (Base)', 'cluster_2', 'checkpoints_cluster_2_base_model'),
    ('Baseline 5)', '01_data/coral_bleaching/reef_support/UNAL_BLEACHING_TAYRONA', 'checkpoints_og_reefsupport'),
]

all_results = []
inference_transform = A.Compose([
    A.Resize(height=INPUT_SIZE[0], width=INPUT_SIZE[1], interpolation=cv2.INTER_NEAREST),
])

print(f"Starting 13-run inference loop using device: {DEVICE}")
print("="*70)

for run_name, dataset_folder, checkpoint_folder in tqdm(inference_runs, desc="Overall Inference Runs"):

    DATASET_ROOT = os.path.join(PROJECT_DIR, dataset_folder)
    CHECKPOINT_PATH = os.path.join(CHECKPOINT_BASE_DIR, checkpoint_folder, 'best_overall_model.pth')

    print(f"\n--- Running: {run_name} (Data: {dataset_folder}) ---")

    if not os.path.isdir(DATASET_ROOT) or not os.path.exists(CHECKPOINT_PATH):
        print(f"Skipping {run_name}: Data or Checkpoint path not found.")
        continue

    try:
        full_dataset = CoralBleachingDataset(root=DATASET_ROOT, transform=inference_transform)

        total_size = len(full_dataset)
        if total_size == 0:
            print(f"Skipping {run_name}: Dataset size is 0.")
            continue

        subset_start_index = math.floor(total_size * 0.85)
        subset_indices = list(range(subset_start_index, total_size))

        inference_subset = Subset(full_dataset, subset_indices)

        print(f"Using the last 15% of images: {len(inference_subset)}/{total_size} images.")

        inference_dataloader = DataLoader(
            inference_subset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )

        model = load_lora_segformer_model(CHECKPOINT_PATH, MODEL_NAME, N_CLASSES, DEVICE)
        if model is None:
            continue

        metrics = run_inference_and_metrics(model, inference_dataloader, DEVICE, N_CLASSES)

        mIoU_weighted, coral_rank_score = calculate_custom_metrics(metrics)

        all_results.append({
            "Run Name": run_name,
            "Accuracy": metrics['Accuracy'],
            "mIoU": metrics['mIoU'],
            "IoU (Background)": metrics['IoU - Background'],
            "IoU (Bleached)": metrics['IoU - Bleached'],
            "IoU (Non-Bleached)": metrics['IoU - Non-Bleached'],
            "Rank Score": coral_rank_score,
            "Test Images": len(inference_subset),
        })

        if len(inference_subset) > 0:
            random_index_in_subset = random.choice(subset_indices)
            visualize_single_sample_canvas(model, full_dataset, random_index_in_subset, run_name, VIS_DIR)

    except FileNotFoundError as e:
        print(f"Skipping {run_name}: File not found error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during {run_name}: {e}")

print("\n" + "="*70)
print("COMPREHENSIVE INFERENCE RESULTS TABLE")
print("="*70)
print(f"Final Canvas Visualizations saved to: {VIS_DIR}")
print("-" * 70)

if all_results:
    results_df = pd.DataFrame(all_results)

    float_cols = results_df.columns.drop(['Run Name', 'Test Images'])
    results_df[float_cols] = results_df[float_cols].apply(lambda x: pd.to_numeric(x, errors='coerce').round(4))

    results_df['Cluster'] = results_df['Run Name'].str.extract('(\d+)').astype(int)

    final_table = results_df.sort_values(by=["Cluster", "Rank Score"], ascending=[True, False])

    final_table = final_table[[
        "Run Name",
        "Test Images",
        "Accuracy",
        "mIoU",
        "IoU (Background)",
        "IoU (Bleached)",
        "IoU (Non-Bleached)",
        "Rank Score",
    ]]

    print(final_table.to_markdown(index=False))
else:
    print("No results were generated. Please check all paths and data contents.")
print("="*70)

<>:114: SyntaxWarning: invalid escape sequence '\d'
<>:114: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-2441536708.py:114: SyntaxWarning: invalid escape sequence '\d'
  results_df['Cluster'] = results_df['Run Name'].str.extract('(\d+)').astype(int)


Final canvas visualizations will be saved to: /content/drive/MyDrive/Data Challenge 3 Group 3/inference_visualization_final_layout
Starting 13-run inference loop using device: cuda


Overall Inference Runs:   0%|          | 0/1 [00:00<?, ?it/s]


--- Running: Cyan 0 (Data: data_cyan_0) ---
Found 256 image-mask pairs for inference.
Using the last 15% of images: 39/256 images.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

pytorch_model.bin:   0%|          | 0.00/99.0M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/98.9M [00:00<?, ?B/s]

Overall Inference Runs: 100%|██████████| 1/1 [00:59<00:00, 59.52s/it]

  -> Saved final canvas visualization for: Cyan 0 (C9_BC_ESb_T3_29nov24_CGomez_corr.jpg)

COMPREHENSIVE INFERENCE RESULTS TABLE
Final Canvas Visualizations saved to: /content/drive/MyDrive/Data Challenge 3 Group 3/inference_visualization_final_layout
----------------------------------------------------------------------
| Run Name   |   Test Images |   Accuracy |   mIoU |   IoU (Background) |   IoU (Bleached) |   IoU (Non-Bleached) |   Rank Score |
|:-----------|--------------:|-----------:|-------:|-------------------:|-----------------:|---------------------:|-------------:|
| Cyan 0     |            39 |     0.8035 | 0.6478 |             0.8318 |           0.6121 |               0.4995 |       0.6274 |
